# Pythonic SysML Elements

This notebook walks through the approach to making SysML elements act Pythonically while in the PyMBE environment in order to support more intuitive codes that inspect and analyze SysML models.

Note that the Markdown headers for code snippets are set up to be translated into docstrings for documentation of the code eventually.

In [ ]:
import json
from dataclasses import dataclass, field
from typing import Any, Dict, List, Set, Tuple, Union
from collections import defaultdict

from uuid import uuid4

with open('example_data/sysml_3a.json', 'r') as fp:
    element_raw_data = json.load(fp)
    
printing_level = "TRACE"

## Interchange data and loading

The API for standing up a model in memory is built around loading data from Python dictionaries. This is mostly based around major APIs of interest (e.g., the SysML v2 REST API) exchanging data in the JSON format with a main set of keys and values.

### Bulk loading model data

The entry point to loading up the models is a bulk load method that gathers collections of dictionaries (each model element is expected to serialize as a dictionary from JSON) and creates a new dictionary that allows for rapid look up of elements by their IDs.

In [ ]:
def index_element_data(elements):
    return {element["@id"]: element for element in elements}

In [ ]:
index_element_data(element_raw_data)[element_raw_data[0]['@id']]

### ID reference format

The serialized JSON files will present IDs as a single entry dictionary of the form {'@id': uuid}.

In [ ]:
def is_id_item(item):
    return isinstance(item, dict) and item['@id'] is not None and isinstance(item['@id'], str)

In [ ]:
is_id_item([])

In [ ]:
is_id_item({'@id': '56851b8c-56c7-4429-8bd0-17fc09b3ce02'})

## Pythonic Model elements

The methods and the class developed here are intended to support the Python representation and manipulation of system model data. The partial methods describe below set up various fields in objects of the Element class for use in downstream or user codes involving this library.

There are a variety of features developed here to make the interaction with system model data more Pythonic.

### Making Items Accessible By Name Rather Than Index

For convenience, a class is introduced that allows for collections of objects to be referenced by name rather than by unique identifier. This requires that names in a given list are unique.

#### Precursors

Classes that this class references are stubbed below for its use.

In [ ]:
class Element():
    """Dummy version for algorithm testing"""
    _data = None
    _model = None

In [ ]:
class Model():
    """Dummy version for algorithm testing"""
    elements: Dict[str, "Element"]

The ListOfNamedItemsClass extends list to make names within model element data a useable index.

In [ ]:
class ListOfNamedItems(list):
    """A list that also can return items by their name."""
    
    # FIXME: Should really build/update the item map on entry modification rather than lookup
    
    # FIXME: figure out why __dir__ of returned objects think they are lists
    def __getitem__(self, key):
        item_map = {
            item._data["name"]: item
            for item in self
            if isinstance(item, Element) and "name" in item._data
        }
        effective_item_map = {
            item._data["effectiveName"]: item
            for item in self
            if isinstance(item, Element) and "effectiveName" in item._data
        }
        if key in item_map:
            if printing_level == "TRACE":
                print(f"Adding {item_map[key]} to list index.")
            return item_map[key]
        if key in effective_item_map:
            if printing_level == "TRACE":
                print(f"Adding {effective_item_map[key]} to list index.")
            return effective_item_map[key]
        if isinstance(key, int):
            return super().__getitem__(key)
        return None

### Navigating fields for owned elements (and more specialized collections)

Elements should be able to directly reference elements through ownership relations. This is done by dynamically constructing Python fields for meta-attributes that start with "owned."

In [ ]:
def set_names_for_owned_for_element(self):
    data = self._data
    
    # iterate through the meta-attributes for the model element in serial form
    for key, items in data.items():
        if printing_level == "TRACE":
            print(f'Key is {key} and items are {items}')
        if key.startswith("owned") and isinstance(items, list):
            new_list = [self._model.elements[item['@id']] for item in items if is_id_item(item)]
            if printing_level == "TRACE" and len(new_list) > 0:
                print(f'Expanding {key} with new list of named items')
            if len(new_list) > 0:
                data[key] = ListOfNamedItems(new_list)

#### Tester

Build an object with only the necessary fields for the test from raw JSON data. In this case, this is an object with "_data" and "_model" fields, and then another object type with an "elements" field.

In [ ]:
def construct_test_element_from_raw(raw_dict, model_object):
    test_element = Element()
    test_element._data = raw_dict
    test_element._model = model_object
    return test_element

In [ ]:
test_model = Model()
indexed_raw = index_element_data(element_raw_data)
test_elements = {indx: construct_test_element_from_raw(raw_item, test_model)
                for indx, raw_item in indexed_raw.items()}
setattr(test_model, "elements", test_elements)

In [ ]:
test_model.elements[element_raw_data[2]['@id']]._data

In [ ]:
set_names_for_owned_for_element(test_model.elements[element_raw_data[2]['@id']])

In [ ]:
test_model.elements[element_raw_data[2]['@id']]._data['ownedElement'][0]._data

In [ ]:
del Element, Model

### Accessing fields Pythonically

The next thing to add to the Element class are magics to match the meta-attributes of loaded elements to properties on Python objects. That is done by adding to the getattr and getitem magics.

This should focus on the metafields with expected literal values in the JSON.

#### More developed Model class

This iteration of the Model class adds the needed methods to support Pythonic relationships.

In [ ]:
class Model:  # pylint: disable=too-many-instance-attributes
    """A SysML v2 Model"""

    # TODO: Look into making elements immutable (e.g., frozen dict)
    elements: Dict[str, "Element"]

    name: str = "SysML v2 Model"

    all_relationships: Dict[str, "Element"] = field(default_factory=dict)
    all_non_relationships: Dict[str, "Element"] = field(default_factory=dict)

    ownedElement: ListOfNamedItems = field(  # pylint: disable=invalid-name
        default_factory=ListOfNamedItems,
    )
    ownedMetatype: Dict[str, List["Element"]] = field(  # pylint: disable=invalid-name
        default_factory=dict,
    )
    ownedRelationship: List["Element"] = field(  # pylint: disable=invalid-name
        default_factory=list,
    )

    max_multiplicity = 100

    source: Any = None

#### More developed Element class

This iteration of the Element class adds the needed methods to support the Pythonic field accessors.

In [ ]:
@dataclass(repr=False)
class Element():
    _data: dict
    _model: Model

    _id: str = field(default_factory=lambda: str(uuid4()))
    _metatype: str = "Element"
    _derived: Dict[str, List] = field(default_factory=lambda: defaultdict(list))
    
    def __getattr__(self, key: str):
        try:
            return self[key]
        except KeyError as exc:
            raise AttributeError(f"Cannot find {key}") from exc
    
    # FIXME: Take advantage of new JSON schema put out in the SysML specification data to isolate fields of most interest.
    
    def __getitem__(self, key: str):
        found = False
        for source in ("_data", "_derived"):
            source = self.__getattribute__(source)
            if key in source:
                if printing_level == 'TRACE':
                    print(f'Found meta-attribute {key}')
                found = True
                item = source[key]
                if printing_level == "TRACE":
                    print(f"(in _getitem_) item is {item}")
                break
        if not found:
            raise KeyError(f"No '{key}' in {self}")

        if isinstance(item, (dict, str)):
            item = self.__safe_dereference(item)
        elif isinstance(item, ListOfNamedItems):
            return item
        elif isinstance(item, (list, tuple, set)):
            items = [self.__safe_dereference(subitem) for subitem in item]
            return type(item)(items)
        return item
    
    def __safe_dereference(self, item):
        """If given a reference to another element, try to get that element"""
        try:
            if isinstance(item, dict) and "@id" in item:
                if len(item) > 1:
                    warn(f"Found a reference with more than one entry: {item}")
                item = item["@id"]
            return self._model.elements[item]
        except KeyError:
            return item

Demonstrate the application of the owned element resolution to all elements in the example data.

In [ ]:
def construct_test_element_from_raw(raw_dict, model_object):
    test_element = Element(_data=raw_dict, _model=model_object, _metatype=raw_dict['@type'])
    test_element._data = raw_dict
    test_element._model = model_object
    return test_element

In [ ]:
with open('example_data/sysml_3a.json', 'r') as fp:
    element_raw_data2 = json.load(fp)

test_model2 = Model()
indexed_raw2 = index_element_data(element_raw_data2)
test_elements2 = {indx: construct_test_element_from_raw(raw_item, test_model2)
                for indx, raw_item in indexed_raw2.items()}
test_model2.elements = test_elements2

In [ ]:
set_names_for_owned_for_element(test_model2.elements[element_raw_data[2]['@id']])

In [ ]:
test_model2.elements[element_raw_data[2]['@id']].ownedElement

In [ ]:
test_model2.elements[element_raw_data[2]['@id']].ownedElement['engineTorque']._data

In [ ]:
printing_level = "ERROR"

In [ ]:
for element_raw in element_raw_data:
    set_names_for_owned_for_element(test_model2.elements[element_raw['@id']])

In [ ]:
test_model2.elements[element_raw_data[2]['@id']]._metatype

In [ ]:
test_model2.elements[element_raw_data[2]['@id']].effectiveName

In [ ]:
test_model2.elements[element_raw_data[2]['@id']]

In [ ]:
[{element_raw['name']: [ele['name']
            for ele in test_model2.elements[element_raw['@id']].ownedElement]}
            for element_raw in element_raw_data]

### Navigating relationships Pythonically

The next Pythonic task for our model elements is the ability to traverse SysML v2 relationships Pythonically, either in the forward (through) or reverse direction with simple property accessors.

In [ ]:
def element_resolve_to_determine_element_as_relationship(self):
    data = self._data
    self._is_relationship = "relatedElement" in data

In [ ]:
resolve_rels = [element_resolve_to_determine_element_as_relationship(ele) for id_, ele in test_model2.elements.items()]

In [ ]:
[(test_model2.elements[ele_data['@id']]._is_relationship,
  test_model2.elements[ele_data['@id']]._data['@type']
 )for ele_data in element_raw_data[1:10]]

In [ ]:
def add_relationship_properties(model, relationship):
    relationship_mapper = {
            "through": ("source", "target"),
            "reverse": ("target", "source"),
        }
    
    endpoints = {
        endpoint_type: [
            model.elements[endpoint["@id"]]
            for endpoint in relationship._data[endpoint_type]
        ]
        for endpoint_type in ("source", "target")
    }
    metatype = relationship._metatype
    for direction, (key1, key2) in relationship_mapper.items():
        endpts1, endpts2 = endpoints[key1], endpoints[key2]
        for endpt1 in endpts1:
            for endpt2 in endpts2:
                endpt1._derived[f"{direction}{metatype}"] += [{"@id": endpt2._data["@id"]}]

In [ ]:
 def model_add_relationship_properties(self):
        self.all_relationships = {
            id_: element for id_, element in self.elements.items() if element._is_relationship
        }
        
        # TODO: make this more elegant...  maybe.
        for relationship in self.all_relationships.values():
            add_relationship_properties(self, relationship)

In [ ]:
model_add_relationship_properties(test_model2)

In [ ]:
test_model2.elements[element_raw_data[11]['@id']]._derived

In [ ]:
test_model2.elements[element_raw_data[11]['@id']].throughOwningMembership[0]._data

### Dealing with derived attributes

In SysML v2 models, there are a large number of derived attributes and collections that can gather the incoming and outgoing relationships to other elements. These are actually the standard means of navigation rather than the relationship-based fields generated specifically for pyMBE (also to make it easier to remember some of the names).

### Creating model elements Pythonically

After setting up a model in memory Pythonically, the next steps will be to create model elements within the model processing session. This method is meant to be bound to the Model class.

In [ ]:
def add_element_to_model(self, element_data):
    element = Element(_data=element_data, _model=self, _metatype=element_data['@type'])
    
    element_resolve_to_determine_element_as_relationship(element)
    if element._is_relationship:
        add_relationship_properties(self, element)
        
    self.elements.update({element_data['@id']: element})

In [ ]:
action_items = [test_model2.elements[element_raw['@id']]
     for element_raw in element_raw_data
     if element_raw['@type'] == 'ActionDefinition' or element_raw['@type'] == 'ActionUsage']

In [ ]:
(action_items[1]['@type'], action_items[0]['@type'])

In [ ]:
new_ft_id = str(uuid4())
feature_typing_data = {
    '@id': new_ft_id,
    '@type': 'FeatureTyping',
    'source': [{'@id': action_items[1]['@id']}],
    'target': [{'@id': action_items[0]['@id']}]
}

In [ ]:
add_element_to_model(test_model2, feature_typing_data)

In [ ]:
test_model2.elements[new_ft_id]._data

In [ ]:
action_items[1].throughFeatureTyping[0]._data